# koelectra 사용 (https://github.com/monologg/KoELECTRA)

Data load / feature selection

일단, '요약문_연구목표', '요약문_연구내용', '요약문_기대효과' 를 사용해보기

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

from tqdm.notebook import tqdm

2021-08-03 11:43:29.058926: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
train_data = pd.read_csv("./data/re_train_data.csv")
test_data = pd.read_csv("./data/re_test_data.csv")

In [4]:
len(train_data), len(test_data)

(174304, 43576)

trainset / validset split

## tokenizer / model settings

In [6]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification
import torch

In [7]:
#model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v2-discriminator")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v2-discriminator")

In [9]:
MAX_LEN = 256

연구목표: ~~128~~ 64 + 연구내용:~~256~~ 128 + 기대효과:~~128~~ 64

각각 반으로 줄임. max_len이 256이 되도록,

remake dataset to fit on BERT / ELECTRA / etc...

In [10]:
def f1_tokenizer(sent, MAX_LEN):
    encoded_dict=tokenizer.encode_plus(text = sent, add_special_tokens=True, max_length=MAX_LEN, pad_to_max_length=True, return_attention_mask=True, truncation = True)
	
    input_id=encoded_dict['input_ids']
    attention_mask=encoded_dict['attention_mask']
    token_type_id = encoded_dict['token_type_ids']
    
    return input_id, attention_mask, token_type_id

def f2_tokenizer(sent, MAX_LEN):
    encoded_dict=tokenizer.encode_plus(text = sent, add_special_tokens=True, max_length=MAX_LEN+1, pad_to_max_length=True, return_attention_mask=True, truncation = True)
	
    input_id=encoded_dict['input_ids'][1:]                   # 각각 index1부터 슬라이스 한 이유는, [CLS] 토큰을 제거하기 위함.
    attention_mask=encoded_dict['attention_mask'][1:]        # [CLS] 토큰을 제거하면 MAX_LEN에서 1만큼 줄어들텐데, 최종 길이를 MAX_LEN으로 하기 위해 
    token_type_id = encoded_dict['token_type_ids'][1:]       # encode_plus(max_length=MAX_LEN+1)
    
    return input_id, attention_mask, token_type_id

위 코드를 짤때, 

# |[CLS] '요약문_연구목표' [SEP] |'요약문_연구내용' [SEP] | '요약문_기대효과' [SEP]|
# |-----       64       -----|-----       128       -----|-----       64       -----|
가 되도록.

In [11]:
input_ids =[]
attention_masks =[]
token_type_ids =[]

for train_obj in tqdm(train_data['요약문_연구목표']):
    input_id1, attention_mask1, token_type_id1 = f1_tokenizer(train_obj, MAX_LEN=64)
    
    
    input_ids.append(input_id1)
    attention_masks.append(attention_mask1)
    token_type_ids.append(token_type_id1)
    
    
for i, train_con in tqdm(enumerate(train_data['요약문_연구내용'])):
    input_id2, attention_mask2, token_type_id2 = f2_tokenizer(train_con, MAX_LEN=128)
    
    input_ids[i].extend(input_id2)
    attention_masks[i].extend(attention_mask2)
    token_type_ids[i].extend(token_type_id2)

for i, train_exp in tqdm(enumerate(train_data['요약문_기대효과'])):
    input_id3, attention_mask3, token_type_id3 = f2_tokenizer(train_exp, MAX_LEN=64)
    
    input_ids[i].extend(input_id3)
    attention_masks[i].extend(attention_mask3)
    token_type_ids[i].extend(token_type_id3)

  0%|          | 0/174304 [00:00<?, ?it/s]

In [12]:
test_input_ids =[]
test_attention_masks =[]
test_token_type_ids =[]

for test_obj in tqdm(test_data['요약문_연구목표']):
    input_id1, attention_mask1, token_type_id1 = f1_tokenizer(test_obj, MAX_LEN=64)
    
    
    test_input_ids.append(input_id1)
    test_attention_masks.append(attention_mask1)
    test_token_type_ids.append(token_type_id1)
    #########################################
    #train_data_labels.append(train_label)
    
for i, test_con in tqdm(enumerate(test_data['요약문_연구내용'])):
    input_id2, attention_mask2, token_type_id2 = f2_tokenizer(test_con, MAX_LEN=128)
    
    test_input_ids[i].extend(input_id2)
    test_attention_masks[i].extend(attention_mask2)
    test_token_type_ids[i].extend(token_type_id2)

for i, test_exp in tqdm(enumerate(test_data['요약문_기대효과'])):
    input_id3, attention_mask3, token_type_id3 = f2_tokenizer(test_exp, MAX_LEN=64)
    
    test_input_ids[i].extend(input_id3)
    test_attention_masks[i].extend(attention_mask3)
    test_token_type_ids[i].extend(token_type_id3)

  0%|          | 0/43576 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [13]:
train_inputs, valid_inputs, train_labels, valid_labels = train_test_split(input_ids, train_data['label'].values, random_state=42, test_size=0.1)
train_masks, valid_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=42, test_size=0.1)
train_token_ids, valid_token_ids, _, _ = train_test_split(token_type_ids, input_ids, random_state=42, test_size=0.1)

In [14]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
train_token_ids = torch.tensor(train_token_ids)
valid_inputs = torch.tensor(valid_inputs)
valid_labels = torch.tensor(valid_labels)
valid_masks = torch.tensor(valid_masks)
valid_token_ids = torch.tensor(valid_token_ids)

In [15]:
BATCH_SIZE = 16

trainset = TensorDataset(train_inputs, train_masks, train_token_ids, train_labels)
train_sampler = RandomSampler(trainset)
train_dataloader = DataLoader(trainset, sampler=train_sampler, batch_size=BATCH_SIZE)

validset = TensorDataset(valid_inputs, valid_masks, valid_token_ids, valid_labels)
valid_sampler = SequentialSampler(validset)
valid_dataloader = DataLoader(validset, sampler=valid_sampler, batch_size=BATCH_SIZE)

In [16]:
test_inputs = torch.tensor(test_input_ids)
# test_labels = torch.tensor(test_data['label'])     test는 non label
test_masks = torch.tensor(test_attention_masks)
test_token_ids = torch.tensor(test_token_type_ids)

testset = TensorDataset(test_inputs, test_masks, test_token_ids)
test_sampler = RandomSampler(testset)
test_dataloader = DataLoader(testset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [17]:
device = torch.device("cuda:0")

In [18]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v2-discriminator", num_labels=46)
model.to(device)

Some weights of the model checkpoint at monologg/koelectra-base-v2-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v2-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32200, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [19]:
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8) # 0으로 나누는 것을 방지하기 위한 epsilon 값
epochs = 10
# 총 훈련 스텝
total_steps = len(train_dataloader) * epochs
# lr 조금씩 감소시키는 스케줄러
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

In [20]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [21]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0
    

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 1000 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_token_ids, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, token_type_ids=b_token_ids, attention_mask=b_input_mask, labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    valid_loss, valid_accuracy = 0, 0
    nb_valid_steps, nb_valid_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in valid_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_token_ids, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, token_type_ids=b_token_ids, attention_mask=b_input_mask, labels=b_labels)
        
        # 로스 구함
        logits = outputs[0]

        valid_loss += logits.item()

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_valid_accuracy = flat_accuracy(logits, label_ids)
        valid_accuracy += tmp_valid_accuracy
        nb_valid_steps += 1
    
    avg_valid_loss = valid_loss / len(valid_dataloader)            
    print("  Accuracy: {0:.2f}".format(valid_accuracy/nb_valid_steps))    
    print("  Average validation loss: {0:.2f}".format(avg_valid_loss))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 10 ========
Training...


RuntimeError: CUDA out of memory. Tried to allocate 192.00 MiB (GPU 6; 10.76 GiB total capacity; 9.55 GiB already allocated; 42.56 MiB free; 9.61 GiB reserved in total by PyTorch)

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
test_loss, test_accuracy = 0, 0
nb_test_steps, nb_test_examples = 0, 0
y_pred_list = []

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_token_ids = batch

    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    y_pred_list.append(outputs)
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))